In [85]:
import pandas as pd
import scipy.signal
import plotly
import plotly.graph_objects as go
import numpy as np

In [86]:
df = pd.read_csv('reumo_final.csv', index_col='Unnamed: 0', dtype=np.float64)
df

,od,ano,valor_m2_vpl,dist_metro
29.0,1.0,1995.0,2038.420646,791.836068
30.0,1.0,1996.0,2092.362073,791.836068
31.0,1.0,1997.0,2101.782314,791.836068
32.0,1.0,1998.0,2106.662125,791.836068
33.0,1.0,1999.0,2105.847268,791.836068
...,...,...,...,...
9757.0,342.0,2015.0,1046.565815,5945.762134
9758.0,342.0,2016.0,1030.352381,5945.762134
9759.0,342.0,2017.0,1024.381486,5945.762134
9760.0,342.0,2018.0,1024.829395,5872.288516


In [87]:
clusters_df = pd.read_csv("../0/od_2017_norm_kmeans3.csv")
clusters_df = clusters_df.apply(pd.to_numeric, errors='coerce')
clusters_df

,Zona,Nome,População,Escolares,Empregos,Área (ha),Per Capita,cluster
0,1,NaN,0.0207,0.1120,0.7111,0.002746,0.2159,0
1,2,NaN,0.0361,0.0252,0.5132,0.005465,0.2393,0
2,3,NaN,0.1085,0.0422,0.1836,0.002296,0.3045,0
3,4,NaN,0.1526,0.0198,0.2232,0.003612,0.5103,1
4,5,NaN,0.0947,0.0304,0.8856,0.003604,0.4287,1
...,...,...,...,...,...,...,...,...
338,339,NaN,0.0000,0.9549,0.2661,0.028109,0.0000,2
339,340,NaN,0.0490,0.0323,0.1346,0.009959,0.6644,1
340,341,NaN,0.0668,0.0655,0.0870,0.008659,0.4633,1
341,342,NaN,0.2261,0.0932,0.1471,0.015386,0.3857,0


In [88]:
clusters_df.columns

Index(['Zona ', 'Nome', 'População', 'Escolares', 'Empregos', 'Área (ha)',
       'Per Capita', 'cluster'],
      dtype='object')

In [89]:
MAP = {}
for idx, row in clusters_df.iterrows():
    id = int(row['Zona '])
    cluster = int(row['cluster'])
    MAP[id] = cluster
MAP

{1: 0,
 2: 0,
 3: 0,
 4: 1,
 5: 1,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 1,
 22: 1,
 23: 0,
 24: 2,
 25: 0,
 26: 0,
 27: 1,
 28: 1,
 29: 1,
 30: 1,
 31: 2,
 32: 1,
 33: 1,
 34: 1,
 35: 0,
 36: 1,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 2,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 1,
 50: 0,
 51: 1,
 52: 1,
 53: 1,
 54: 1,
 55: 1,
 56: 1,
 57: 1,
 58: 0,
 59: 1,
 60: 1,
 61: 1,
 62: 1,
 63: 1,
 64: 1,
 65: 1,
 66: 1,
 67: 1,
 68: 1,
 69: 1,
 70: 1,
 71: 1,
 72: 1,
 73: 1,
 74: 1,
 75: 1,
 76: 1,
 77: 1,
 78: 1,
 79: 1,
 80: 1,
 81: 1,
 82: 1,
 83: 1,
 84: 1,
 85: 1,
 86: 1,
 87: 1,
 88: 1,
 89: 1,
 90: 1,
 91: 0,
 92: 0,
 93: 2,
 94: 1,
 95: 1,
 96: 1,
 97: 1,
 98: 0,
 99: 0,
 100: 1,
 101: 1,
 102: 1,
 103: 1,
 104: 1,
 105: 1,
 106: 0,
 107: 1,
 108: 0,
 109: 2,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 2,
 117: 2,
 118: 0,
 119: 2,
 120: 0,
 121: 2,
 122: 0,
 123: 2,
 

In [90]:
# iterate over the df lines 
for index, row in df.iterrows():
    # get the cluster of the current line
    try:
        zona = int(row['od'])
        cluster = MAP[zona]
    except KeyError:
        print("Unable to find cluster for index: ", index, " - zone ", zona)
        continue
    # print it
    # print(cluster)
    # set the cluster value in the df
    df.loc[index, 'cluster'] = int(cluster)


In [91]:
df['cluster'].value_counts()

cluster
0.0    4750
1.0    2050
2.0    1593
Name: count, dtype: int64

In [92]:
anos = list(df['ano'].unique())
collection = {}

for ano in anos:
    # Create a copy of the filtered DataFrame to avoid SettingWithCopyWarning
    df_ano = df[df['ano'] == ano].sort_values(by=['dist_metro'], ascending=True).copy()
    x = df_ano['dist_metro']
    y = df_ano['valor_m2_vpl']
    
    # Ensure the window length is odd and less than the size of y
    window = min(299, len(y) - (1 if len(y) % 2 == 0 else 0))
    order = 3
    
    # Apply the Savitzky-Golay filter
    try:
        yhat = scipy.signal.savgol_filter(y, window, order)
        # Add the filtered data as a new column
        df_ano['valor_m2_vpl_filtrado'] = yhat
        # Add to the collection
        collection[ano] = df_ano
    except ValueError as e:
        # Handle the error if the window size is larger than the data size
        print(f"Error for year {ano}: {e}")
    
    # The break statement was here; if you want to process only the first year, keep it.
    # If you want to process all years, remove it.
    # break

anos.sort()


In [93]:
clusters = list(df['cluster'].unique())
clusters

[0.0, 1.0, 2.0]

In [94]:
df_ano['cluster'].value_counts()

cluster
0.0    190
1.0     82
2.0     64
Name: count, dtype: int64

In [95]:
df_ano

,od,ano,valor_m2_vpl,dist_metro,cluster,valor_m2_vpl_filtrado
53.0,1.0,2019.0,3658.913651,791.836068,0.0,4236.346121
169.0,5.0,2019.0,3578.753775,845.990935,1.0,4206.793683
198.0,6.0,2019.0,4106.604257,931.580154,0.0,4177.386622
140.0,4.0,2019.0,2487.264422,961.386980,1.0,4148.124584
82.0,2.0,2019.0,3504.998307,975.757001,0.0,4119.007214
...,...,...,...,...,...,...
3388.0,117.0,2019.0,19.567454,17840.029796,2.0,9.887268
3417.0,118.0,2019.0,10.891874,17960.287577,0.0,1.230704
8427.0,296.0,2019.0,2.358132,18871.347473,0.0,-7.512718
8485.0,298.0,2019.0,4.542697,22356.001581,2.0,-16.344028


In [101]:
# plotar valor_m2_vpl e valor_m2_vpl_filtrado num mesmo grafico do plotly

for ano in anos:
    # print(ano)
    fig = go.Figure()
    
    df_ano = collection[ano]
    # print(df_ano.shape)
    # force the data to be sorted by dist_metro
    df_ano = df_ano.sort_values(by=['dist_metro'], ascending=True)
    
    x = df_ano['dist_metro']
    y = df_ano['valor_m2_vpl']
    yhat = df_ano['valor_m2_vpl_filtrado']
    # # print(len(x), len(y))
    clusters = list(df_ano['cluster'].unique())

    for cluster in clusters:
        # Corrected filtering here
        df_cluster = df_ano[df_ano['cluster'] == cluster]
        x_cluster = df_cluster['dist_metro']
        y_cluster = df_cluster['valor_m2_vpl']
        # print(len(x_cluster), len(y_cluster))
        fig.add_scatter(x=x_cluster, y=y_cluster, mode='markers', name=f'Cluster {cluster}', marker=dict(size=15))

    # add a line for filtered # change color to black
    fig.add_trace(go.Scatter(x=x, y=yhat, mode='lines', name='Filtered',line=dict(width=4), marker=dict(color='black', size=20)))
    # add title with the ano
    fig.update_layout(title_text=f'{int(ano)}')
    # set the xlim: 0 - 30000
    fig.update_xaxes(range=[0, 30000])
    # set the ylim: 0 - 10000
    fig.update_yaxes(range=[0, 10000])

    # set the xlabel: "Distância média até as 10 estações mais próximas"
    fig.update_xaxes(title_text='Distance to the 10th closest metro stations (m)')
    # set the ylabel: "Valor das construções por metro quadrado da zona (R$/m²)"
    fig.update_yaxes(title_text='Construction value per square meter (R$/m²)')

    # align the title to the center and increase font to 50
    fig.update_layout(title_x=0.5, title_font_size=50)

    # adjust height and width
    fig.update_layout(height=600, width=1000)

    # put the legend inside the plot
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

    # increase font size
    fig.update_layout(font_size=20)

    # show the plot
    # fig.show()

    # save to png
    fig.write_image(f'plots/clusters_{int(ano)}.png')